In [1]:
# Cell 1 - Imports, environment setup and choose experiment

import os
import sys

import torch
from sklearn.metrics import classification_report

# --------- detect if running on Colab ---------
try:
    import google.colab  # type: ignore
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

if IN_COLAB:
    from google.colab import drive  # type: ignore
    drive.mount("/content/drive")

    # Path to your project on Google Drive
    PROJECT_ROOT = "/content/drive/MyDrive/[2025-2026] AN2DL/challenge2"
else:
    # Local project root (adjust if needed)
    PROJECT_ROOT = os.getcwd()  # or a fixed path

# Change working directory to project root and make sure we can import "includes"
os.chdir(PROJECT_ROOT)
if PROJECT_ROOT not in sys.path:
    sys.path.append(PROJECT_ROOT)

print("Project root:", PROJECT_ROOT)
print("Working dir:", os.getcwd())
print("Python path contains 'includes'? ->", "includes" in os.listdir("."))

# --------- now we can import from includes ---------
from includes.config import EXPERIMENTS
from includes.data_utils import (
    load_labels_and_split,
    get_transforms,
    create_dataloaders,
)
from includes.model_utils import (
    build_model,
    create_criterion_optimizer_scheduler,
    train_model,
    evaluate,
)
from includes.inference_utils import (
    create_test_loader,
    run_inference_and_save,
)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# --------- choose which experiment to run ---------
# available:
#   "baseline"
#   "resnet50_big"
#   "resnet50_big_masks"
#   "challenge_2-2"
EXP_NAME = "effb0_224_maskmul_f1"
cfg = EXPERIMENTS[EXP_NAME]


# Override project_root and out_dir according to environment
cfg.project_root = PROJECT_ROOT

# Save submissions to:
#   local:  <PROJECT_ROOT>/out
#   Colab:  /content/drive/.../challenge2/out
cfg.out_dir = "out"

cfg


Project root: /home/federico/Desktop/Shared/Projects/ANN
Working dir: /home/federico/Desktop/Shared/Projects/ANN
Python path contains 'includes'? -> True
Using device: cpu


TrainingConfig(exp_name='effb0_224_maskmul_f1', project_root='/home/federico/Desktop/Shared/Projects/ANN', data_dir='data', out_dir='out', train_img_dir='train_data', test_img_dir='test_data', labels_csv='train_labels.csv', mask_dir=None, backbone='efficientnet_b0', img_size=224, batch_size=16, num_workers=4, val_size=0.2, random_seed=42, mask_mode='multiply', lr=0.0001, weight_decay=0.0001, epochs=50, use_scheduler=True, use_masks=True)

In [2]:
# Cell 2 - Dataframes and loaders

train_df, val_df, unique_labels, label_to_idx, idx_to_label = load_labels_and_split(cfg)
print("Labels:", unique_labels)

train_t, val_t = get_transforms(cfg)
train_loader, val_loader = create_dataloaders(cfg, train_df, val_df, train_t, val_t)


Labels: ['HER2(+)', 'Luminal A', 'Luminal B', 'Triple negative']


In [3]:
# Cell 3 - Model, criterion, optimizer, scheduler

model = build_model(cfg, num_classes=len(unique_labels), device=device)
criterion, optimizer, scheduler = create_criterion_optimizer_scheduler(
    cfg, model, train_df, device
)


In [4]:
# Cell 4 - Training

best_state, history = train_model(
    cfg, model, train_loader, val_loader, criterion, optimizer, scheduler, device
)
model.load_state_dict(best_state)

# Final report on validation
val_loss, val_acc, y_true, y_pred = evaluate(model, val_loader, criterion, device)
print("Val acc:", val_acc)
print(classification_report(y_true, y_pred, target_names=unique_labels))


Starting training - experiment: effb0_224_maskmul_f1
Backbone: efficientnet_b0  |  img_size: 224  |  epochs: 50

--------------------------------------------------------------
[Epoch 1/50]
    [Batch  71/71] loss=1.3783  f1=0.2937
  Train - loss: 1.3783  |  f1: 0.2937
  Val   - loss: 1.3820  |  f1: 0.2562
  >> New best model! val_f1 improved to 0.2562

--------------------------------------------------------------
[Epoch 2/50]


KeyboardInterrupt: 

In [ ]:
# Cell 5 - Inference and submission

test_loader, test_files = create_test_loader(cfg, val_t)
run_inference_and_save(cfg, model, test_loader, idx_to_label, device, output_csv="submission_modular.csv")
